<center><h1> Parsing </h1></center>

# 1. Import packages

In [1]:
import requests
import re
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
import time
import sys
from stem import Signal
from stem.control import Controller

from tqdm import tqdm_notebook
from fake_useragent import UserAgent

# 2. Parsing functions

## 2.1 Get link functions

Буду парсить сайт Эхо Москвы

In [2]:
main_link = 'https://echo.msk.ru'
news_link = main_link+'/news'

In [3]:
def get_soup(page_link):
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    soup = BeautifulSoup(response.content, "html.parser")
    return soup

In [4]:
def get_news_links(page_number):
    cur_day=(datetime.today() - timedelta(days=page_number))
    main_link = 'https://echo.msk.ru'
    news_link = main_link+'/news'
    page_link =news_link+'/'+str(cur_day.year)+'/'+ str(cur_day.month)+'/'+ str(cur_day.day)+'.html'
    soup = get_soup(page_link)
    news_links = soup.findAll("div", attrs={'class':'preview newsblock iblock'})
    news_links = [news_link.find('a', attrs={'class':'view'})['href'] for news_link in news_links]
    news_links = list(set(news_links))
    return news_links

In [5]:
echo_links = get_news_links(1)

In [6]:
print(main_link + echo_links[0])

https://echo.msk.ru/news/2646337-echo.html


## 2.2 Grab data functions

In [7]:
def _get_title(news_soup):
    title = news_soup.find("h1")    
    title = None if not title else title.text.strip()
    return title

def _get_views_comments_num(news_soup):
    comments = news_soup.find("a", attrs={'class':'comm'})
    comments = comments.find('span', attrs={'class':'count'})
    comments = None if not comments else int(comments.text)
    views = news_soup.find("a", attrs={'class':'view'})
    views = views.find('span', attrs={'class':'count'})
    views = None if not views else int(views.text)
    return (views, comments)

def _get_publication_date_and_time(news_soup):
    date = news_soup.find("div", attrs={'class':'date'})
    date = date.find("span", class_='')
    date = None if not date else date.text.strip()
    
    cur_time = news_soup.find("div", attrs={'class':'date'})
    cur_time = cur_time.find("strong", attrs={'class':'red'})
    cur_time = None if not cur_time else cur_time.text.strip()
    return (date+' '+cur_time)

def _get_main_text(news_soup):
    text_main = news_soup.find(
        'span', 
        attrs={'class':'_ga1_on_ include-relap-widget contextualizable'}
    )
    text_main = text_main.findAll('p')
    if not text_main:
        return None
    
    text_main = [paragraph.text.strip() for paragraph in text_main]
    text_main = " ".join(text_main)
    return text_main

def get_news_page_info(news_link):
    news_soup = get_soup(main_link + news_link)
    
    views, comments = _get_views_comments_num(news_soup)
    title = _get_title(news_soup)
    publication_date = _get_publication_date_and_time(news_soup)
    main_text = _get_main_text(news_soup)
    
    info = {
        'views':views,
        'comments':comments,
        'title':title,
        'publication_date':publication_date,
        'main_text':main_text
    }
    
    return info

In [8]:
len(echo_links)

195

In [9]:
news_dataset = pd.DataFrame(columns=['views', 'comments', 'title', 'publication_date', 'main_text'])

for news_link in tqdm_notebook(echo_links):
    news_dataset = news_dataset.append(get_news_page_info(news_link), ignore_index=True)
    time.sleep(0.3)

In [10]:
news_dataset.head(10)

,views,comments,title,publication_date,main_text
0,1004,11,Порошенко назвал записи его разговоров с полит...,20 мая 2020 22:06,Экс-президент Украины Петр Порошенко назвал за...
1,506,1,Различные меры по борьбе с коронавирусом с сег...,20 мая 2020 06:04,"Если в одних ограничения смягчают, то в других..."
2,608,2,У сети алкомаркетов «Красное и белое» могут от...,20 мая 2020 22:05,Причиной стали нарушения требований профилакти...
3,1228,9,Командующий антиправительственными силами в Ли...,20 мая 2020 10:34,Это сделано по случаю мусульманского праздник...
4,3442,42,Чехия официально отказалась передавать России ...,20 мая 2020 09:32,Ранее он был демонтирован в Праге. Как пояс...
5,1190,8,В Карелии врачам запретили публично комментиро...,20 мая 2020 17:04,Соответствующий приказ регионального Минздрава...
6,1780,15,Палестина выходит из всех соглашений с Израиле...,20 мая 2020 11:32,Глава автономии Махмуд Аббас обвинил Иерусал...
7,2085,15,Пользователи приложения для контроля за больны...,20 мая 2020 22:02,Такие данные привели столичные власти. Речь ид...
8,3945,12,Газопровод «Северный поток» не попадет под огр...,20 мая 2020 21:05,Такое постановление приняло Федеральное сетево...
9,3240,10,В результате стрельбы на юго-востоке Москвы ра...,20 мая 2020 20:35,В результате стрельбы на юго-востоке Москвы ра...


In [11]:
news_dataset.tail(10)

,views,comments,title,publication_date,main_text
185,2305,35,Путин поддержал идею о запуске регулярных пост...,20 мая 2020 19:31,Владимир Путин поддержал идею о запуске регуля...
186,454,6,Зам.министра промышленности Виктор Евтухов: Ме...,20 мая 2020 11:05,"Медицинские маски будут дешеветь в аптеках, —..."
187,60,0,Чемпионат Коста-Рики станет первым футбольным ...,20 мая 2020 06:08,Чемпионат Коста-Рики станет первым профессиона...
188,320,0,Количество тестирований на коронавирус в Дагес...,20 мая 2020 00:05,"Планируется, что в начале следующей недели это..."
189,1385,1,"Сестры Хачатурян, которых обвиняют в убийстве ...",20 мая 2020 10:32,"Сестры Хачатурян, которых обвиняют в убийстве ..."
190,4435,6,В Москве ежедневно госпитализируют около тысяч...,20 мая 2020 13:02,По этим показателям столица вышла на уровень ...
191,710,3,Жители Москвы могут обжаловать штрафы за нар...,20 мая 2020 12:32,"Для этого здесь запущен специальный раздел, —..."
192,2493,9,В Россию могут разрешить ввозить дефицитные по...,20 мая 2020 22:33,Соответствующий законопроект внесен в Госдуму....
193,7502,15,"РБК пишет, что в Кремле хотят провести голосов...",20 мая 2020 07:31,Издание ссылается на четыре источника. Ранее о...
194,931,13,Совет Федерации сегодня предложит правительств...,20 мая 2020 08:06,В верхней палате парламента решили вернуться к...


In [12]:
news_dataset.isna().sum()

views               0
comments            0
title               0
publication_date    0
main_text           0
dtype: int64

In [13]:
news_dataset = pd.DataFrame(columns=['views', 'comments', 'title', 'publication_date', 'main_text'])

for page_number in tqdm_notebook(range(0, 6)):
    news_links = get_news_links(page_number)
    for news_link in tqdm_notebook(news_links, leave=False):
        news_dataset = news_dataset.append(get_news_page_info(news_link), ignore_index=True)
        time.sleep(0.3)

In [14]:
news_dataset.head()

,views,comments,title,publication_date,main_text
0,1080,2,"Войска ливийского правительства, признанного О...",21 мая 2020 06:33,Как сообщает агентство РИА Новости со ссылкой ...
1,679,3,В жилых домах и социальных объектах Подмосковь...,21 мая 2020 06:37,"В последние дни в Московской области, также, к..."
2,189,1,Олимпийские игры в Токио придется полностью от...,21 мая 2020 07:31,Об этом в интервью телерадиокомпании ВВС сказа...
3,1775,1,В мире за последние сутки зафиксирован наиболь...,21 мая 2020 06:31,Глава Всемирной организации здравоохранения Те...
4,160,1,В Свердловской области губернатор пригрозил от...,21 мая 2020 06:49,"По словам Евгения Куйвашева, в ходе проверки в..."


In [15]:
news_dataset.isna().sum()

views               0
comments            0
title               0
publication_date    0
main_text           0
dtype: int64

Сайт неплохо парсится. У них нет автоблокировок, но в учебных целях настрою Tor.  
А вот у сайта Риановости каждая ссылка на новую страницу новостей в ленте генерится каким-то PRBS генератором.

# 3. Using TOR 

## 3.1 Import packages

In [16]:
import socks
import socket
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket

## 3.2 Chek IP and Chenge IP

In [17]:
def checkIP():
    ip = requests.get('http://checkip.dyndns.org').content
    soup = BeautifulSoup(ip, 'html.parser')
    print(soup.find('body').text)

In [18]:
checkIP()

Current IP Address: 185.220.100.244


In [19]:
for i in range(10):
    checkIP()
    time.sleep(10)

Current IP Address: 185.220.100.244
Current IP Address: 185.220.101.208
Current IP Address: 185.220.102.6
Current IP Address: 94.230.208.148
Current IP Address: 46.19.141.83
Current IP Address: 162.247.74.200
Current IP Address: 51.38.162.232
Current IP Address: 149.202.80.194
Current IP Address: 23.129.64.192
Current IP Address: 95.128.43.164


In [20]:
news_dataset = pd.DataFrame(columns=['views', 'comments', 'title', 'publication_date', 'main_text'])

for page_number in tqdm_notebook(range(0, 6)):
    news_links = get_news_links(page_number)
    for news_link in tqdm_notebook(news_links, leave=False):
        news_dataset = news_dataset.append(get_news_page_info(news_link), ignore_index=True)
        time.sleep(0.3)

## 3.3 Final parsing

In [ ]:
final_df = pd.DataFrame(columns=['views', 'comments', 'title', 'publication_date', 'main_text'])


for page_number in tqdm_notebook(range(0,1455)):
    # собрали хрефы с текущей страницы
    news_links = get_news_links(page_number)  
    for news_links in tqdm_notebook(news_links, leave=False):
        # иногда с первого раза страничка не парсится
        for i in range(5):
            try:
                # пытаемся собрать по мему немного даты
                data_row = get_news_page_info(news_links)           
                # и закидываем её в таблицу
                final_df = final_df.append(data_row, ignore_index=True)  
                # если всё получилось - выходим из внутреннего цикла
                break
            except:
                # Иначе, пробуем еще несколько раз, пока не закончатся попытки
                continue
                
    final_df.to_csv('NEWS_{}.csv'.format(page_number))
    if page_number//10:
        print(final_df.shape)
final_df.to_csv('NEWS_full.csv')

Тут я задал очень большое число новостей. Скрипт успешно парсил всю ночь и отработал на 10%. Решил не парсить сайт до конца, так как это может занять несколько дней. 

# 4. Parsing results

Всего распарсил 136 дней в новостной ленте.

In [21]:
data = pd.read_csv("NEWS_135.csv")

In [22]:
data.head()

,Unnamed: 0,views,comments,title,publication_date,main_text
0,0,1415,8,Япония снова заявила о претензиях на Южные Курилы,19 мая 2020 12:06,В докладе главы японского МИДа острова Кунашир...
1,1,105,0,В Риме после карантина открываются первые музеи,19 мая 2020 06:08,Посещение будет возможно только по предварител...
2,2,5579,30,В Краснодарском крае сотрудников «скорой» Абин...,19 мая 2020 16:33,В Краснодарском крае сотрудников станции скоро...
3,3,577,4,Власти Псковской области отрицают нехватку сре...,19 мая 2020 00:08,Власти Псковской области отрицают нехватку сре...
4,4,3517,8,В Москве четвертый день подряд фиксируют более...,19 мая 2020 06:48,"Скончался еще 71 пациент, сообщили ночью в опе..."


In [23]:
data.isna().sum()

Unnamed: 0           0
views                0
comments             0
title                0
publication_date     0
main_text           11
dtype: int64

Есть новости без текста, но с заголовками.

In [24]:
data.shape

(25692, 6)

In [25]:
data['title'].value_counts()

Горздрав Екатеринбурга подало иск к «Эху Москвы в Екатеринбурге» из-за высказываний Ройзмана о коронавирусе           110
«Медиазона»: В России от коронавируса умерли не меньше 186 медиков                                                    110
Министр труда и соцзащиты заявил, что семьи с двумя детьми могут получить суммарно более 100 тысяч рублей              98
В Краснодарском крае сотрудников «скорой» Абинской больницы вызвали в полицию после их жалобы на отсутствие выплат     85
Совет Федерации предлагает ограничить продажу табака и алкоголя                                                        30
                                                                                                                     ... 
Путин потребовал выплатить всем медработникам до 15 мая надбавки к зарплате за работу с заболевшими коронавирусом       1
Социальные фонды из-за распространения эпидемии могут потерять более 2 триллионов рублей                                1
В Россию с сегодняшнего 

  
  
**Пока я парсил, страницы обновляись, и поэтому значения заголовка повторяются.**

In [26]:
data=data.drop_duplicates(keep='last', subset='title')

Дропнул все строки с повторяющимися заголовками.

In [27]:
data.shape

(24458, 6)

Стало новостей поменьше.

In [28]:
data.head(10)

,Unnamed: 0,views,comments,title,publication_date,main_text
0,0,1415,8,Япония снова заявила о претензиях на Южные Курилы,19 мая 2020 12:06,В докладе главы японского МИДа острова Кунашир...
1,1,105,0,В Риме после карантина открываются первые музеи,19 мая 2020 06:08,Посещение будет возможно только по предварител...
3,3,577,4,Власти Псковской области отрицают нехватку сре...,19 мая 2020 00:08,Власти Псковской области отрицают нехватку сре...
4,4,3517,8,В Москве четвертый день подряд фиксируют более...,19 мая 2020 06:48,"Скончался еще 71 пациент, сообщили ночью в опе..."
5,5,3897,21,На Украине разразился скандал после интервью ж...,19 мая 2020 18:27,С ним поговорил Дмитрий Гордон. Интервью возму...
6,6,1191,1,В России возбуждено первое уголовное дело из-з...,19 мая 2020 20:01,Его завели на Кубани после видеообращения сот...
7,7,973,6,В Подмосковье совершил жесткую посадку военный...,19 мая 2020 22:05,Вертолет Ми-8 ВКС России совершил жесткую поса...
8,8,2469,0,Следственный комитет проводит проверку по ф...,19 мая 2020 20:04,Там в результате взрыва воспламеняющейся жид...
9,9,704,16,Глава Роснано Анатолий Чубайс считает зависимо...,19 мая 2020 13:04,"Государственный чиновник напомнил о том, что н..."
10,10,209,3,Премьер-министр Михаил Мишустин завершил лечен...,19 мая 2020 14:09,Премьер-министр Михаил Мишустин завершил лечен...


In [29]:
data.tail(10)

,Unnamed: 0,views,comments,title,publication_date,main_text
25682,25682,1138,15,Столтенберг: Новый конфликт на Ближнем Востоке...,06 января 2020 20:30,Новый конфликт на Ближнем Востоке никому не вы...
25683,25683,3338,4,Убитый в Подмосковье Сергей Хайкин работал в Н...,06 января 2020 17:31,К самой госкорпорации он отношения не имел. Об...
25684,25684,794,1,Борьба за пост спикера оппозиционной Националь...,06 января 2020 02:03,Часть депутатов избрали в качестве нового пред...
25685,25685,1235,9,Цены на нефть на фоне обострения на Ближнем Во...,06 января 2020 19:00,Нефть марки Brent сегодня торговалась в Лондон...
25686,25686,601,7,ЕС будет принимать решение о возможных сан...,06 января 2020 16:31,Об этом заявил официальный представитель внешн...
25687,25687,1677,25,Израильские власти утвердили планы строительст...,06 января 2020 23:00,Израильские власти утвердили планы строительст...
25688,25688,4460,27,"Дональд Трамп пригрозил Ираку санкциями, если ...",06 января 2020 10:00,Резкую реакцию американского лидера вызвало ре...
25689,25689,1584,0,Картина Сэма Мендеса «1917» получила «Золотой ...,06 января 2020 13:30,Картина Сэма Мендеса «1917» получила «Золотой ...
25690,25690,3435,11,В Подмосковье обнаружили мертвыми сотрудника «...,06 января 2020 16:01,Следственный комитет возбудил дело по статье «...
25691,25691,315,0,Полиция в Ростове-на-Дону задержала подозревае...,06 января 2020 02:03,"При нем обнаружили средства самообороны, говор..."
